In [1]:
import tensorflow as tf  
import numpy as np       
import pandas as pd       
from matplotlib import pyplot as plt   
import seaborn as sns  
import time          
from tensorflow.keras.callbacks import History, ReduceLROnPlateau, TensorBoard       
from tensorflow.keras import metrics          
from sklearn.metrics import confusion_matrix          
import pickle
from tensorflow.keras.applications import MobileNet
from tensorflow.keras.layers import GlobalAveragePooling2D, Reshape, multiply, add, Dense, Conv2D, GlobalMaxPooling2D, Activation
from tensorflow.keras import backend as K
import platform
import os
from tensorflow.keras.layers import GlobalAveragePooling2D, Multiply, Dense
from keras.layers import GlobalAveragePooling2D, Dense, Multiply
import platform
import os

In [2]:
# Display Settings
pd.options.display.max_columns = None
pd.options.display.max_rows = None
sns.set()

In [3]:
# System Information Print the Python version
print("Python version: ", platform.python_version())
print("TensorFlow version: ", tf.__version__)
print("Current working directory: ", os.getcwd())
!uname -a
!nvidia-smi

Python version:  3.11.7
TensorFlow version:  2.15.0
Current working directory:  /root/.jupyter/张彤/消融实验
Linux w3q2ulc9.vm 5.15.0-60-generic #66-Ubuntu SMP Fri Jan 20 14:29:49 UTC 2023 x86_64 x86_64 x86_64 GNU/Linux
Sun Oct 27 16:11:08 2024       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 545.23.08              Driver Version: 545.23.08    CUDA Version: 12.3     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  NVIDIA GeForce RTX 4090        On  | 00000000:09:00.0 Off |                  Off |
|  0%   24C    P8               8W

In [4]:
# Hyperparameter Settings
EPOCHS = 50
IMAGE_SIZE = (224, 224)
IMAGE_PATH = "../data"
LEARNING_RATE = 1e-4
BATCH_SIZE = 64

In [5]:
# Load Dataset
train_ds = tf.keras.utils.image_dataset_from_directory(
    IMAGE_PATH,
    validation_split=0.2,
    subset="training",    
    seed=123,
    image_size=IMAGE_SIZE,
    batch_size=BATCH_SIZE
)

vaild_ds = tf.keras.utils.image_dataset_from_directory(
    IMAGE_PATH,
    validation_split=0.2,
    subset="validation",
    seed=123,
    image_size=IMAGE_SIZE,
    batch_size=BATCH_SIZE
)

Found 14080 files belonging to 100 classes.
Using 11264 files for training.


2024-10-27 16:11:10.564041: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:901] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2024-10-27 16:11:10.564423: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:901] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2024-10-27 16:11:10.680285: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:901] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-

Found 14080 files belonging to 100 classes.
Using 2816 files for validation.


In [6]:
# Image Normalization
mean = [0.485, 0.456, 0.406]
std = [0.229, 0.224, 0.225]

def normalize_image(image):
    return (image - mean_tensor) / std_tensor


In [7]:
# Data Augmentation
train_image_augment = tf.keras.Sequential([
    tf.keras.layers.Rescaling(1 / 255.0),
    tf.keras.layers.RandomRotation(factor=0.2),
    tf.keras.layers.RandomFlip(),
])

valid_image_augment = tf.keras.Sequential([
    tf.keras.layers.Rescaling(1 / 255.0),
])

In [8]:
# Input Processing
def process_train_input(images, labels):
    return train_image_augment(images), labels

def process_valid_input(images, labels):
    return valid_image_augment(images), labels


In [9]:
# Data Preprocessing
def convert_types_and_encode(x, y):
    y = tf.cast(y, tf.int32)  
    y_one_hot = tf.one_hot(y, 100)  
    return x, y_one_hot

In [10]:
# Apply Data Preprocessing
train_ds = train_ds.map(convert_types_and_encode)
train_ds = train_ds.map(process_train_input, num_parallel_calls=tf.data.AUTOTUNE)
train_ds = train_ds.prefetch(tf.data.AUTOTUNE)

vaild_ds = vaild_ds.map(convert_types_and_encode)
vaild_ds = vaild_ds.map(process_valid_input, num_parallel_calls=tf.data.AUTOTUNE)
vaild_ds = vaild_ds.prefetch(tf.data.AUTOTUNE)

In [11]:
# Define the ECA layer
def eca_layer(input_tensor, k_size=3):
    # Apply average pooling across the spatial dimensions
    avg_pool = K.mean(input_tensor, axis=[1, 2], keepdims=True)
    # Apply Conv2D instead of Conv1D for 2D spatial data
    conv = Conv2D(filters=1, kernel_size=(k_size, 1), padding='same', activation='sigmoid')(avg_pool)
    # Multiply the attention map with the input feature maps
    weighted_input = Multiply()([input_tensor, conv])  
    return weighted_input

# Create MobileNet base model
base_model = MobileNet(input_shape=(224, 224, 3), include_top=False, weights='imagenet')
feature_maps = base_model.output

# Add ECA layer
eca_feature_maps = eca_layer(feature_maps)

# Add global average pooling
x = GlobalAveragePooling2D()(eca_feature_maps)

# Add fully connected layers
x = Dense(1024, activation='relu')(x)
outputs = Dense(100, activation='softmax')(x)

# Create the final model
model = tf.keras.Model(inputs=base_model.input, outputs=outputs)

# Compile the model
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])


In [12]:
# Compile Model
optimizer = tf.keras.optimizers.Adam(learning_rate=LEARNING_RATE)
loss_fn = tf.keras.losses.CategoricalCrossentropy()
metrics = [
    tf.keras.metrics.CategoricalAccuracy(),
    tf.keras.metrics.Precision(),
    tf.keras.metrics.Recall(),
]

model.compile(optimizer=optimizer, loss=loss_fn, metrics=metrics)

In [13]:
# Train Model
log_dir = "../Running result/3/3"
tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=log_dir, histogram_freq=1)
lr_scheduler = tf.keras.callbacks.ReduceLROnPlateau(monitor='val_categorical_accuracy', factor=0.5, patience=2, min_lr=1e-8)

history = model.fit(train_ds, epochs=EPOCHS, validation_data=vaild_ds, callbacks=[lr_scheduler, tensorboard_callback])

Epoch 1/50


2024-10-27 16:11:25.791755: I external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:454] Loaded cuDNN version 8904
2024-10-27 16:11:26.010528: I external/local_tsl/tsl/platform/default/subprocess.cc:304] Start cannot spawn child process: No such file or directory
2024-10-27 16:11:27.202888: I external/local_xla/xla/service/service.cc:168] XLA service 0xc9b9570 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
2024-10-27 16:11:27.202963: I external/local_xla/xla/service/service.cc:176]   StreamExecutor device (0): NVIDIA GeForce RTX 4090, Compute Capability 8.9
2024-10-27 16:11:27.202985: I external/local_xla/xla/service/service.cc:176]   StreamExecutor device (1): NVIDIA GeForce RTX 4090, Compute Capability 8.9
2024-10-27 16:11:27.226287: I tensorflow/compiler/mlir/tensorflow/utils/dump_mlir_util.cc:269] disabling MLIR crash reproducer, set env var `MLIR_CRASH_REPRODUCER_DIRECTORY` to enable.
I0000 00:00:1730016687.479263    1701 device_compiler.

176/176 [==============================] - 34s 79ms/step - loss: 2.5141 - categorical_accuracy: 0.4374 - precision: 0.8895 - recall: 0.2173 - val_loss: 1.5374 - val_categorical_accuracy: 0.5916 - val_precision: 0.8379 - val_recall: 0.3782 - lr: 1.0000e-04
Epoch 2/50
176/176 [==============================] - 13s 70ms/step - loss: 0.8843 - categorical_accuracy: 0.7648 - precision: 0.9090 - recall: 0.6255 - val_loss: 0.8338 - val_categorical_accuracy: 0.7702 - val_precision: 0.8945 - val_recall: 0.6506 - lr: 1.0000e-04
Epoch 3/50
176/176 [==============================] - 13s 72ms/step - loss: 0.5623 - categorical_accuracy: 0.8478 - precision: 0.9271 - recall: 0.7651 - val_loss: 0.6488 - val_categorical_accuracy: 0.8143 - val_precision: 0.9093 - val_recall: 0.7404 - lr: 1.0000e-04
Epoch 4/50
176/176 [==============================] - 13s 71ms/step - loss: 0.3928 - categorical_accuracy: 0.8949 - precision: 0.9451 - recall: 0.8379 - val_loss: 0.5514 - val_categorical_accuracy: 0.8487 - val

In [14]:
import pandas as pd  # Import Pandas library
import os

# Define save path
model_dir = "../Running result/3/3"
excel_file_path = os.path.join(model_dir, "3.xlsx")  # Path to save the Excel file
model_file_path = os.path.join(model_dir, "3.h5")  # Path to save the model

# Check if the save directory exists; create it if it does not
if not os.path.exists(model_dir):
    os.makedirs(model_dir)

# Save training history to an Excel file
history_df = pd.DataFrame(history.history)  
history_df.to_excel(excel_file_path, index=False)
print(f"Training history saved to: {excel_file_path}")

# Save the model to the specified path
model.save(model_file_path)
print(f"Model saved to: {model_file_path}")


Training history saved to: ../Running result/3/3/3.xlsx


/root/anaconda3/lib/python3.11/site-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


Model saved to: ../Running result/3/3/3.h5
